In [ ]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
import numpy as np
import time
import math
import matplotlib.pyplot as plt
from numpy import interp
import fileinput
import sympy as sp
import struct
import random
from tqdm import tqdm

#SE=Overlay("./hw/SYSTEM.bit")
SE=Overlay("./hw2/SYSTEM.bit")

In [ ]:
# IP Registers and MMIO Addressing

NEUROPROCESSOR_BASE_ADDR        = 0x43C00000
ADDRESS_RANGE                   = 0xFFFF
ADDRESS_OFFSET                  = 0x10   # READ/WRITE
PRE_SYNAPTIC_STREAM_ROUTER	    = 0xE0   # READ/WRITE
POST_SYNAPTIC_STREAM_ROUTER	    = 0xE4   # READ/WRITE
PRE_SYNAPTIC_STREAM_STROBE	    = 0xE8   # READ/WRITE
POST_SYNAPTIC_STREAM_STROBE	    = 0xEC   # READ/WRITE
POST_SYNAPTIC_STREAM_DATACOUNT  = 0xF4   # READ/WRITE
SPIKE_STREAM_STROBE	            = 0xF0   # READ/WRITE
POST_SYN_FIFO_RST	            = 0x00   # READ/WRITE
HYPERCOLUMN_RST	                = 0x04   # READ/WRITE
NMC_STATE_RST	                = 0x08   # READ/WRITE
NMC_HARD_RST	                = 0x0C   # READ/WRITE
PRE_SYN_FIFO_RST	            = 0x10   # READ/WRITE
DC_RST_VECTOR	                = 0x14   # READ/WRITE
SPIKE_BUFFER_RST_VECTOR	        = 0x18   # READ/WRITE
POST_SYN_FIFO_EMPTY	            = 0x1C   # READ ONLY
POST_SYN_FIFO_FULL	            = 0x20   # READ ONLY
PRE_SYN_FIFO_EMPTY	            = 0x2C   # READ ONLY
PRE_SYN_FIFO_FULL	            = 0x30   # READ ONLY
SPIKE_BUFFER_FULL	            = 0x3C   # READ ONLY
SPIKE_BUFFER_EMPTY	            = 0x40   # READ ONLY
NMC_SPIKE_FLAGS	                = 0x44   # READ ONLY
NMC_MEM_VIOLATION	            = 0x48   # READ ONLY
NMC_MATH_ERROR	                = 0x4C   # READ ONLY
NMC_FINISHED	                = 0x50   # READ ONLY
NMC_XN_LOW	                    = 0x54   # RW
NMC_XN_HIGH	                    = 0x58   # RW
NMC_COLD_START	                = 0x5C   # RW
HYPERCOLUMN_0_INFERENCE_CYCLES	= 0x60   # RW
HYPERCOLUMN_1_INFERENCE_CYCLES	= 0x64   # RW
HYPERCOLUMN_2_INFERENCE_CYCLES	= 0x68   # RW
HYPERCOLUMN_3_INFERENCE_CYCLES	= 0x6C   # RW
HYPERCOLUMN_INFERENCE_START	    = 0x70   # RW
HYPERCOLUMN_INFERENCE_DONE	    = 0x74   # READ ONLY
HYPERCOLUMN_LEARNING_START	    = 0x78   # RW
HYPERCOLUMN_LEARNING_DONE	    = 0x7C   # READ ONLY
AXON_CONTROLS_SET_FEEDBACK	    = 0x80   # RW
PRE_SYN_MUX		                = 0x84   # RW
NMC_PRE_SYN_INTERFACE_CNTRL		= 0x88   # RW
NMC_PROG_MEM_WENABLE	        = 0x8C   # RW
NMC_XX_NEURON_ID	            = 0x90   # RW
NMC_XX_LAST_SPIKE_TIME	        = 0x94   # RW
NMC_XX_SYN_QFACTOR	            = 0x98   # RW
NMC_XX_PF_LOW_ADDR	            = 0x9C   # RW
NMC_XX_NMODEL_REFRACTORY_DUR	= 0xA0   # RW
NMC_XX_NMODEL_NPARAM_DATA	    = 0xA4   # RW
NMC_XX_NMODEL_NPARAM_ADDR	    = 0xA4   # RW
NMC_XX_NMODEL_ULEARN_LUT_TABLE	= 0xAC   # RW
NMC_XX_NMODEL_ULEARN_LUT_EN	    = 0xB4   # RW
NMC_XX_NMODEL_WMAX_WMIN_LR	    = 0xB8   # RW
POST_SYN_MUX	                = 0xBC   # RW
NMC_POST_SYN_INTERFACE_CNTRL	= 0xC0   # RW
NMC_XX_R_NMODEL_NEURON_ID		= 0xC4   # READ ONLY
NMC_XX_R_NNMODEL_NEW_SPIKE_TIME	= 0xC8   # READ ONLY
NMC_XX_R_NMODEL_SYN_QFACTOR		= 0xCC   # READ ONLY
NMC_XX_R_NMODEL_PF_LOW_ADDR		= 0xD0   # READ ONLY
NMC_XX_R_NMODEL_REFRACTORY_DUR  = 0xD4   # READ ONLY
NMC_XX_R_NMODEL_NPARAM_DATAOUT  = 0xD8   # READ ONLY
NMC_XX_REDIST_NMODEL_DADDR		= 0xDC   # RW
INFERENCE_PERFORMANCE           = 0x34   # READ ONLY
SPIKE_PERFORMANCE               = 0xFC   # READ ONLY
POST_SYN_PERFORMANCE            = 0xF8   # READ ONLY
PRE_SYN_PERFORMANCE             = 0x28   # READ ONLY
NMC_PERFORMANCE                 = 0x24   # READ ONLY
TRAINING_PERFORMANCE            = 0x38   # READ ONLY


In [ ]:
# Neuron Model Assembly code to NMC executable code.

def NModelAssembler(InputStringArray, ArrayBaseAddress):

    def lw_binary_from_string(command):
        parts = command.split(',')  
        reg_num = int(parts[1][1])  
        num = int(parts[2]) 
        reg_bin = format(reg_num, '03b')  
        num_bin = format(num, '09b') 
        binary = f"0001{reg_bin}{num_bin}" 
        return binary[-16:] 

    def sw_binary_from_string(command):
        parts = command.split(',')  
        reg_num = int(parts[1][1])  
        num = int(parts[2]) 
        reg_bin = format(reg_num, '03b')  
        num_bin = format(num, '09b')  
        binary = f"0010{reg_bin}{num_bin}"  
        return binary[-16:]  

    def fmac_binary_from_string(command):
        parts = command.split(',') 
        reg1_num = int(parts[1][1]) 
        reg2_num = int(parts[2][1]) 
        reg1_bin = format(reg1_num, '03b') 
        reg2_bin = format(reg2_num, '03b') 
        binary = f"0100{'000000'}{reg1_bin}{reg2_bin}"  
        return binary

    def smac_binary_from_string(command):
        parts = command.split(',')  
        reg1_num = int(parts[1][1]) 
        reg2_num = int(parts[2][1]) 
        reg1_bin = format(reg1_num, '03b')  
        reg2_bin = format(reg2_num, '03b')  
        binary = f"0101{'000000'}{reg1_bin}{reg2_bin}"  
        return binary

    def clracc_binary_from_string(command):
        binary = "0110000000000000"  
        return binary

    def comp_binary_from_string(command):
        parts = command.split(',') 
        reg1_num = int(parts[1][1])  
        reg2_num = int(parts[2][1])  
        reg1_bin = format(reg1_num, '03b')  
        reg2_bin = format(reg2_num, '03b') 
        binary = f"0111{'000000'}{reg1_bin}{reg2_bin}" 
        return binary

    def spk_binary_from_string(command):
        binary = "1011000000000000" 
        return binary

    def getacc_binary_from_string(command):
        parts = command.split(',')  
        register_num = int(parts[1][1])  
        reg_bin = format(register_num, '03b')  
        binary = f"0011{reg_bin}000000000" 
        return binary

    def return_binary_from_string(command):
        binary = "1101000000000000" 
        return binary

    def strf_binary_from_string(command):
        parts = command.split(',')  
        num = int(parts[1])  
        num_bin = format(num, '08b')  
        binary = f"11100000{num_bin}" 
        return binary

    def bil_binary(num):
        num_bin = format(num, '010b')  
        binary = f"1000{'00'}{num_bin}"  
        return binary

    def big_binary(num):
        num_bin = format(num, '010b')  
        binary = f"1010{'00'}{num_bin}"  
        return binary

    def bie_binary(num):
        num_bin = format(num, '010b')  
        binary = f"1001{'00'}{num_bin}"  
        return binary

    def convert_to_binary(command):
        if command.startswith('lw'):
            return lw_binary_from_string(command)
        elif command.startswith('sw'):
            return sw_binary_from_string(command)
        elif command.startswith('fmac'):
            return fmac_binary_from_string(command)
        elif command.startswith('smac'):
            return smac_binary_from_string(command)
        elif command.startswith('clr'):
            return clracc_binary_from_string(command)
        elif command.startswith('com'):
            return comp_binary_from_string(command)
        elif command.startswith('spk'):
            return spk_binary_from_string(command)
        elif command.startswith('get'):
            return getacc_binary_from_string(command)
        elif command.startswith('ret'):
            return return_binary_from_string(command)
        elif command.startswith('str'):
            return strf_binary_from_string(command)
        elif command.startswith('bil'):
            num = int(command[4:])
            return bil_binary(num)
        elif command.startswith('big'):
            num = int(command[4:])
            return big_binary(num)
        elif command.startswith('bie'):
            num = int(command[4:])
            return bie_binary(num)
        else:
            return "Invalid command"
        
    commands = InputStringArray.split('\n')
    NMC_Executable = []

    for command in commands:
        if command.strip():
            binary_representation = convert_to_binary(command)
            NMC_Executable.append(binary_representation)

    NMC_Binary = np.array([int(binary, 2) for binary in NMC_Executable], dtype=np.uint16)

    NMC_INT = NMC_Binary.astype(int)
    NMC_PROGRAM_DATA = []

    for i in range(len(NMC_INT)):
        NMC_PROGRAM_DATA.append(NMC_INT[i].item())

    SRM_ADDR = range(ArrayBaseAddress, ArrayBaseAddress + len(NMC_PROGRAM_DATA))
    SRM_ADDR_UINT32 = np.array(SRM_ADDR, dtype=int)

    SRM_ADDRESS_SPACE = []
    for i in range(len(SRM_ADDR_UINT32)):
        SRM_ADDRESS_SPACE.append(SRM_ADDR_UINT32[i].item())

    NMC_PROGRAM_DATA = []

    for i in range(len(SRM_ADDRESS_SPACE)):
        NMC_PROGRAM_DATA.append(NMC_INT[i].item() << 16 | SRM_ADDRESS_SPACE[i])

    return NMC_PROGRAM_DATA

In [ ]:
def ULEARN_FIT(TimewindowLeft, TimewindowRight, ZeroPoint, Samples, expr1, expr2):
    t = sp.symbols('t')
    equation1 = sp.sympify(expr1)
    equation2 = sp.sympify(expr2)

    leftHandSpace = np.linspace(-TimewindowLeft, ZeroPoint, int(Samples/2))
    rightHandSpace = np.linspace(ZeroPoint, TimewindowRight, int(Samples/2))
    synapticUpdateFunctionLHS = np.zeros(int(Samples/2))
    synapticUpdateFunctionRHS = np.zeros(int(Samples/2))

    for i in range(0, len(leftHandSpace)):
        synapticUpdateFunctionLHS[i] = equation1.subs(t, leftHandSpace[i])
    
    for i in range(0, len(rightHandSpace)):
        synapticUpdateFunctionRHS[i] = equation2.subs(t, rightHandSpace[i])

    synapticFunction = np.concatenate((synapticUpdateFunctionLHS, synapticUpdateFunctionRHS), axis=None)
    timespace = np.concatenate((leftHandSpace, rightHandSpace), axis=None)

    qFactor = 2 / math.pow(2, 8)
    qSynapticFunction = synapticFunction / qFactor
    mSynapticFunction = qSynapticFunction.astype(int)
    qTimeWindow = np.linspace(-int(Samples/2), int(Samples/2) - 1, int(Samples))

    result = mSynapticFunction
    
    result_array=result.astype(np.int8)
    
    return result_array

# Create ULEARN Table

def LearningWindowMapper(LearningWindow):
    OnPreAddressSpace = np.arange(-128, 1, 1, dtype=np.int8)
    OnPostAddressSpace = np.arange(1, 128, dtype=np.int8)
    CompleteAddressSpace = np.concatenate((OnPreAddressSpace, OnPostAddressSpace))

    combined_binary = [bin(x & 0xFF)[2:].zfill(8) + bin(y & 0xFF)[2:].zfill(8) for x, y in zip(LearningWindow, CompleteAddressSpace)]

    ULEARN_REG = []
    ULEARN_REG_UINT16 = np.array([np.uint16(int(b, 2)) for b in combined_binary], dtype=np.uint16)

    for i in range(len(ULEARN_REG_UINT16)):
        ULEARN_REG.append(ULEARN_REG_UINT16[i].astype(int))

    ULEARN_DATA = np.array(ULEARN_REG, dtype=int)
    ULEARN_MAP = []

    for i in range(len(ULEARN_DATA)):
        ULEARN_MAP.append(ULEARN_DATA[i].item())

    return ULEARN_MAP

def Synapse(TimewindowLeft, TimewindowRight, ZeroPoint, Samples, expr1, expr2):
    SModel = ULEARN_FIT(TimewindowLeft, TimewindowRight, ZeroPoint, Samples, expr1, expr2)
    SModelTable = LearningWindowMapper(SModel)
    return SModelTable


def CreateRandomSynapse(length):
    return np.random.randint(0, 255, size=length, dtype=np.uint8)

def InitializeTimestamps(array):
    temp = array.astype(np.uint16)
    temp = temp << 8
    temp = temp | 0xFF
    return temp 

def Synapse2Stream(*arrays):
    uint64_arrays = [array.astype(np.uint64) for array in arrays]

    shifted_arrays = [np.left_shift(array, shift) for array, shift in zip(uint64_arrays, (48, 32, 16, 0))]

    result = np.bitwise_or.reduce(shifted_arrays)

    return result

def Stream2Synapse(result):
    shifts = (48, 32, 16, 0)
    
    reverse_shifted_arrays = [np.right_shift(result, shift) for shift in shifts]
    
    uint16_arrays = [array.astype(np.uint16) for array in reverse_shifted_arrays]
    
    return tuple(uint16_arrays)

def count_lines(file_path):
    try:
        with fileinput.input(files=(file_path,)) as file:
            line_count = sum(1 for _ in file)
        return line_count
    except FileNotFoundError:
        return 0

def ParameterAddressMerger(NPARAM,NADDR):
    MergedData = (NPARAM << 16) | NADDR
    return MergedData

In [ ]:
# SEMEION DATASET READ and Convolution Functions

#FilePath = "./semeion/semeion.data"  

# Pulling images from dataset

def PullImagesFromDataset(file_path):
    lines = []
    with open(file_path, 'r') as file:
        for line in file:
            lines.append(line.strip())  

    return lines

# Lines to grayscale images

def Line2Image(line):
    elements = line.split(" ")
    image_elements = [int(element) for element in elements[:256]]

    image = np.array(image_elements).reshape(16, 16)

    return image

# Convolution Kernel and Process
    
def ReceptiveField(InputImage):
    c1 =  0.625
    c2 =  0.125
    c3 = -0.125
    c4 = -.5
    
    ImagePixels= 16

    #Receptive field kernel
    w = [[c4 ,c3 , c2 ,c3 ,c4],
        [c3 ,c2 , c1 ,c2 ,c3],
        [c4 ,c1 , 1 ,c1 ,c2],
        [c3 ,c2 , c1 ,c2 ,c3],
        [c4 ,c3 , c2 ,c3 ,c4]]

    MembraneMap = np.zeros([ImagePixels,ImagePixels])
    Range = [-2,-1,0,1,2]
    xOffset = 2
    yOffset = 2

    #Convolution
    for i in range(ImagePixels):
        for j in range(ImagePixels):
            ConvSum = 0
            for m in Range:
                for n in Range:
                    if (i+m)>=0 and (i+m)<=ImagePixels-1 and (j+n)>=0 and (j+n)<=ImagePixels-1:
                        ConvSum = ConvSum + w[xOffset+m][yOffset+n]*InputImage[i+m][j+n]/255
            MembraneMap[i][j] = ConvSum
    return MembraneMap
    plt.imshow(Image, cmap="gray")
    plt.show()
    
# Convolution result to Rate Coded Spike Train
    
def Image2SpikeTrain(Image,TimeUnits,ImagePixels):

    SpikeTrain = []

    for l in range(ImagePixels):
        for m in range(ImagePixels):
        
            temp = np.zeros([(TimeUnits+1),])

            freq = interp(Image[l][m], [np.min(Image),np.max(Image)], [1,50])
            
            time_period = math.ceil(TimeUnits/freq)

            time_of_spike = time_period
            if(Image[l][m]>0):
                while time_of_spike<(TimeUnits+1):
                    temp[int(time_of_spike)] = 1
                    time_of_spike+= time_period
            SpikeTrain.append(temp)
    return SpikeTrain

# Convert Spike Train into an 2D Array for Packaging

def SpikeTrainOpt(InputSpikeSequence):
    
    temp2d = np.array(InputSpikeSequence)

    Out2D = temp2d.astype(np.uint8)

    return Out2D

# Pack Spikes into an UINT16 vector.

def SpikePacker(matrix):
    rows, columns = matrix.shape
    target_length = columns * (rows // 16)
    result = np.zeros(target_length, dtype=np.uint16)

    index = 0
    for j in range(columns):
        for i in range(0, rows, 16):
            value = 0
            for k in range(16):
                value |= (matrix[i + k, j] << (15 - k)) 
            result[index] = value
            index += 1

    return result

def WeightReconstructor(weights, pixel_x):
    weights = np.array(weights)
    weights = np.reshape(weights, (pixel_x,pixel_x))
    img = np.zeros((pixel_x,pixel_x))
    ImagePixels= 16
    for i in range(pixel_x):
        for j in range(pixel_x):
            img[i][j] = int(interp(weights[i][j], [-1,1], [-128,127]))	
    return img

def CompileImageDataset(Dataset,ImagePixels,RateCodeTime,Filename):
    
    Lines = PullImagesFromDataset(Dataset)
    TimeUnits = RateCodeTime
    ImagePixels = ImagePixels
    
    file_name = Filename

    for i in range(min(256, len(Lines))):
        Image = Line2Image(Lines[i])
        #if EnableFiltering == True:
            #ConvResult = ReceptiveField(Image)
        #SpikeTrain = Image2SpikeTrain(ConvResult,TimeUnits,ImagePixels)
        SpikeTrain = Image2SpikeTrain(Image,TimeUnits,ImagePixels)
        OptimizedSpikeTrain = SpikeTrainOpt(SpikeTrain)
        HypercolumnSpikeStream = SpikePacker(OptimizedSpikeTrain)
        with open(file_name, 'a') as file:
            np.savetxt(file, [HypercolumnSpikeStream], fmt='%d', delimiter=' ', newline='\n')

In [ ]:
def ExtractSynapses(array_uint16):
    shape = array_uint16.shape
    array_uint8 = np.zeros(shape, dtype=np.uint8)
    for index, value in np.ndenumerate(array_uint16):
        extracted_value = np.uint8(value >> 8) 
        array_uint8[index] = extracted_value 
    return array_uint8

def ExtractTimestamps(array_uint16):
    shape = array_uint16.shape
    array_uint8 = np.zeros(shape, dtype=np.uint8)
    for index, value in np.ndenumerate(array_uint16):
        extracted_value = np.uint8(value & 0xFF) 
        array_uint8[index] = extracted_value
    return array_uint8

In [ ]:
# Read Converted SEMION Spikes
# Read Packed Image Data

def DatasetLine2SpikeArray(filename, line_number):
    try:
        with open(filename, 'r') as file:
            for _ in range(line_number - 1):
                file.readline()
            line = file.readline().strip().split()
            uint16_array = np.array([int(value) for value in line], dtype=np.uint16)
            return uint16_array
    except FileNotFoundError:
        print(f"{filename} file not found.")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None



In [ ]:
# Hardware Reset

def HWReset():
    mmio.write(PRE_SYN_FIFO_RST,0xFFFF)
    mmio.write(POST_SYN_FIFO_RST,0xFFFF)
    mmio.write(HYPERCOLUMN_RST,0xFFFF)
    mmio.write(NMC_STATE_RST,0xFFFF)
    mmio.write(NMC_HARD_RST,0xFFFF)
    mmio.write(DC_RST_VECTOR,0xFFFF)
    mmio.write(SPIKE_BUFFER_RST_VECTOR,0xFFFF)
    mmio.write(PRE_SYN_FIFO_RST,0x0)
    mmio.write(POST_SYN_FIFO_RST,0x0)
    mmio.write(HYPERCOLUMN_RST,0x0)
    mmio.write(NMC_STATE_RST,0x0)
    mmio.write(NMC_HARD_RST,0x0)
    mmio.write(DC_RST_VECTOR,0x0)
    mmio.write(SPIKE_BUFFER_RST_VECTOR,0x0)

# NMC Program Memory Access

def LoadNeuronModel(NModelProgramData):

    mmio.write(NMC_XN_LOW,0x0)         # NMC Execute-Never Region Low Address  = 0x0
    mmio.write(NMC_XN_HIGH,0x80)       # NMC Execute-Never Region High Address = 0x80 

    mmio.write(NMC_PRE_SYN_INTERFACE_CNTRL,0xFFFF) # Enable All NMC Pre-Synaptic Interface
    mmio.write(NMC_PROG_MEM_WENABLE,0xFFFF)           # Take over All NMC program memory

    for i in range(len(NModelProgramData)):
        mmio.write(NMC_XX_NMODEL_NPARAM_DATA,NModelProgramData[i])   # Write Neuron Program to all NMC units

    mmio.write(NMC_PROG_MEM_WENABLE,0x0)        # Give back NMC program memory to all NMC units
    mmio.write(NMC_PRE_SYN_INTERFACE_CNTRL,0x0) # Disable NMC Pre-Synaptic Interfaces
    
# LEARNING ENGINE PROGRAMMING

def LearningTablePass(LearningTable,LearningRate):

    mmio.write(NMC_XX_NMODEL_ULEARN_LUT_EN,0xFFFF) # Enable ULEARN Look-up Tables
    
    if isinstance(LearningTable, (list, tuple)):
    
        for i in range(0,len(LearningTable)):
            mmio.write(NMC_XX_NMODEL_ULEARN_LUT_TABLE,LearningTable[i])   # Pass the ULEARN_LUT_DATA + ULEARN_LUT_ADDR to all ULEARN Units in all HYPERCOLUMN units
        mmio.write(NMC_XX_NMODEL_ULEARN_LUT_EN,0x0) # Disable ULEARN Look-up Tables
        
    else:
        
        mmio.write(NMC_XX_NMODEL_ULEARN_LUT_TABLE,LearningTable)   # Pass the ULEARN_LUT_DATA + ULEARN_LUT_ADDR to all ULEARN Units in all HYPERCOLUMN units
        mmio.write(NMC_XX_NMODEL_ULEARN_LUT_EN,0x0) # Disable ULEARN Look-up Tables
        
    # Wmax = 1.0 (127 = 7F)
    # Wmin = -1.0 (-127 = 81)
    
    LRQ = 2/255
    TempLR = LearningRate/LRQ
    TempT = math.ceil(TempLR)
    LRate = np.uint32(TempT)
    
    ULEARNParamReg = int(0x007F8100|LRate) 
    mmio.write(NMC_XX_NMODEL_WMAX_WMIN_LR,ULEARNParamReg)
    
# NMC Parameter Pass
    
def NMCParamPass(Data,Address,NMCUnits):
    
    mmio.write(NMC_PRE_SYN_INTERFACE_CNTRL,NMCUnits)
    mmio.write(NMC_PROG_MEM_WENABLE,NMCUnits)
    
    if isinstance(Data, (list, tuple)):
        
        for DatIndx in range (len(Data)):
            NMCData = ParameterAddressMerger(Data[DatIndx], Address[DatIndx])
            mmio.write(NMC_XX_NMODEL_NPARAM_DATA,NMCData)
            
    else:
        
        NMCData = ParameterAddressMerger(Data,Address)
        mmio.write(NMC_XX_NMODEL_NPARAM_DATA,NMCData)

    mmio.write(NMC_PROG_MEM_WENABLE,0x0)        # Give back NMC program memory to all NMC units
    mmio.write(NMC_PRE_SYN_INTERFACE_CNTRL,0x0) # Disable NMC Pre-Synaptic Interface
       

In [ ]:
# Neuron Container Class. It only contains neuron's program flow low address, last spike time, parameter array, refractory duration, synapses,
# learning table for NMC units, AxonMap for spike propogation to other neurons.

class Neuron:
    def __init__(self,NeuronID,ProgramFlowLowAddress,LastSpikeTime,SynapticQFactor,NeuronParamDataArray,NeuronParamAddrArray,NeuronRefractoryDuration,Synapses,SynapseType,AxonMap,LearningRate):

        self.NeuronID = NeuronID
        self.ProgramFlowLowAddress = ProgramFlowLowAddress
        self.LastSpikeTime = LastSpikeTime
        self.SynapticQFactor = SynapticQFactor
        self.NeuronParamDataArray = NeuronParamDataArray
        self.NeuronParamAddrArray = NeuronParamAddrArray
        self.NeuronRefractoryDuration = NeuronRefractoryDuration
        self.Synapses = Synapses
        self.SynapseType = SynapseType
        self.AxonMap = AxonMap
        self.LearningRate = LearningRate

 

In [ ]:
# DMA Engine Initialization

PreSynapticStream = SE.PRE_SYN_DMA
PostSynapticStream = SE.POST_SYN_DMA
SpikeStream = SE.SPIKE_DMA            

In [ ]:
# DMA Engines And Streaming Interfaces

PushSynapses = PreSynapticStream.sendchannel
PullSynapses = PostSynapticStream.recvchannel
PushSpikes = SpikeStream.sendchannel

# IP Interface Initialization

mmio = MMIO(NEUROPROCESSOR_BASE_ADDR,ADDRESS_RANGE)

In [ ]:
class Network():
    def __init__(self, input_size):
        self.input_data = []
        self.input_size = input_size
        self.output_data = []
        self.LayerSizes = []
        self.NeuronCount = 0
        self.LayerSizes.append(input_size)
        self.neurons = []  
        self.ParameterSpace = []
        self.SpikeSpace = []
        self.TotalInferenceTime = 0
        self.TotalNMCExecutionTime = 0
        self.TotalTrainingTime = 0
        
    def Append(self, NMODELLowAddr, ParamData, ParamAddr, SynapseType, LearningRate, LayerSize, Scheme, Probability):
        if self.NeuronCount + LayerSize > 16384:
            raise ValueError("Network neuron limit 16384 is exceeded.")
        if LayerSize > 4096:
            raise ValueError("Layer size cannot exceed 4096.")          
        if not 0 <= LearningRate <= 1:
            raise ValueError("Learning Rate must be between 0 and 1")       
        if Scheme not in ['fc', 'random']:
            raise ValueError("Unrecognized connection scheme.") 

        self.NeuronCount += LayerSize
        self.LayerSizes.append(LayerSize)
        
        neurons = []  
        for NeuronIndex in range(1, LayerSize + 1):
            neuron = Neuron(NeuronIndex, ProgramFlowLowAddress=NMODELLowAddr, LastSpikeTime=0x7F,
                            SynapticQFactor=0x2004, NeuronParamDataArray=ParamData,
                            NeuronParamAddrArray=ParamAddr, NeuronRefractoryDuration=0,
                            Synapses=None, SynapseType=SynapseType, AxonMap=None, LearningRate=LearningRate)
            if Scheme == 'fc':
                synapsearray = CreateRandomSynapse(self.LayerSizes[-2])
                synaptic_data = InitializeTimestamps(synapsearray)
                neuron.Synapses = synaptic_data
            elif Scheme == 'random':
                                
                synapsearray = CreateRandomSynapse(self.LayerSizes[-2])
                synaptic_data = InitializeTimestamps(synapsearray)
                    
                for SynapseIndex in range(self.LayerSizes[-2]):
                    connect_probability = random.random() 

                    if Probability >= connect_probability:
                        synaptic_data[SynapseIndex] = synaptic_data[SynapseIndex]
                    else:
                        synaptic_data[SynapseIndex] = 0   
                        
                neuron.Synapses = synaptic_data
                
            neurons.append(neuron)
        
        self.neurons.extend(neurons)
                       
    def Evaluate(self,InputFeed,EnableLearning):
        
        mmio.write(PRE_SYN_FIFO_RST,0xFFFF)
        mmio.write(POST_SYN_FIFO_RST,0xFFFF)
        mmio.write(HYPERCOLUMN_RST,0xFFFF)
        mmio.write(NMC_STATE_RST,0xFFFF)
        mmio.write(DC_RST_VECTOR,0xFFFF)
        mmio.write(PRE_SYN_FIFO_RST,0x0)
        mmio.write(POST_SYN_FIFO_RST,0x0)
        mmio.write(HYPERCOLUMN_RST,0x0)
        mmio.write(NMC_STATE_RST,0x0)
        mmio.write(DC_RST_VECTOR,0x0)
        
        OutStream = []
        
        if not isinstance(EnableLearning, bool):
            raise ValueError("EnableLearning parameter must be a boolean value")
        
        for LayerIndex in range(1,len(self.LayerSizes)):

            if LayerIndex == 1:
                
                PreSpikeStream=InputFeed
                SpikeCycle = math.ceil(self.input_size/16)
                
                if len(InputFeed) != SpikeCycle:
                    raise ValueError("Spike vector length and input layer doesn't match")
                    
                NeuronIndexOffset = 0
                    
            else:
                
                PreSpikeStream=OutStream
                SpikeCycle = math.ceil(self.LayerSizes[LayerIndex]/16)
                
                NeuronIndexOffset = NeuronIndexOffset+self.LayerSizes[LayerIndex-1]
            
            PostSpikeStream = []

            for NeuronInLayer in range(0, self.LayerSizes[LayerIndex], 16):
                
                IndexDif = self.LayerSizes[LayerIndex]-NeuronInLayer
                if IndexDif >=16:
                    PackOffset = 16
                else:
                    PackOffset = IndexDif
                    
                Package = self.neurons[NeuronIndexOffset+NeuronInLayer:NeuronIndexOffset+NeuronInLayer+PackOffset]
                
                PushAllSpikes = (np.uint64(PreSpikeStream) << np.uint64(48)) | (np.uint64(PreSpikeStream) << np.uint64(32)) | (np.uint64(PreSpikeStream) << np.uint64(16)) | np.uint64(PreSpikeStream)
                SpikeBuffer = allocate(shape=(len(PushAllSpikes),), dtype=np.uint64)
                
                mmio.write(HYPERCOLUMN_0_INFERENCE_CYCLES,int(SpikeCycle))
                mmio.write(HYPERCOLUMN_1_INFERENCE_CYCLES,int(SpikeCycle))
                mmio.write(HYPERCOLUMN_2_INFERENCE_CYCLES,int(SpikeCycle))
                mmio.write(HYPERCOLUMN_3_INFERENCE_CYCLES,int(SpikeCycle))                   
            
                for l in range(len(PushAllSpikes)):
                    SpikeBuffer[l] = PushAllSpikes[l]
 
                PushSpikes.transfer(SpikeBuffer)
                PushSpikes.wait()
                
                del SpikeBuffer
                    
                for m in range(len(Package)): 
                    mmio.write(NMC_PRE_SYN_INTERFACE_CNTRL, int(2**m))
                    mmio.write(PRE_SYN_MUX, m) 
                    mmio.write(NMC_XX_NEURON_ID, Package[m].NeuronID)    
                    mmio.write(NMC_XX_LAST_SPIKE_TIME, Package[m].LastSpikeTime)
                    mmio.write(NMC_XX_SYN_QFACTOR, Package[m].SynapticQFactor)
                    mmio.write(NMC_XX_PF_LOW_ADDR, Package[m].ProgramFlowLowAddress)
                    mmio.write(NMC_XX_NMODEL_REFRACTORY_DUR, Package[m].NeuronRefractoryDuration)
                    NMCParamPass(Package[m].NeuronParamDataArray,Package[m].NeuronParamAddrArray,2**m)
                        
                    if EnableLearning == True:
                        LearningTablePass(Package[m].SynapseType,Package[m].LearningRate)
                    else:
                        LearningTablePass(0x0,0x0)
           
                HYPERCOLUMN0_PreSynapticStream = []
                HYPERCOLUMN1_PreSynapticStream = []
                HYPERCOLUMN2_PreSynapticStream = []
                HYPERCOLUMN3_PreSynapticStream = []   
                NMCVector = 0
    
                if len(Package) == 1:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses)
                    NMCVector = 0x0001
                            
                elif len(Package) == 2:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses)
                    NMCVector = 0x0003
                        
                elif len(Package) == 3:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses)
                    NMCVector = 0x0007
                        
                elif len(Package) == 4:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses, Package[3].Synapses)
                    NMCVector = 0x000F
                        
                elif len(Package) == 5:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses, Package[3].Synapses)
                    HYPERCOLUMN1_PreSynapticStream = Synapse2Stream(Package[4].Synapses)     
                    NMCVector = 0x001F
                    LearningVector = 0x001F
                                                       
                elif len(Package) == 6:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses, Package[3].Synapses)
                    HYPERCOLUMN1_PreSynapticStream = Synapse2Stream(Package[4].Synapses , Package[5].Synapses)   
                    NMCVector = 0x003F
                    
                elif len(Package) == 7:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses, Package[3].Synapses)
                    HYPERCOLUMN1_PreSynapticStream = Synapse2Stream(Package[4].Synapses , Package[5].Synapses, Package[6].Synapses)
                    NMCVector = 0x007F
                        
                elif len(Package) == 8:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses, Package[3].Synapses)
                    HYPERCOLUMN1_PreSynapticStream = Synapse2Stream(Package[4].Synapses , Package[5].Synapses, Package[6].Synapses, Package[7].Synapses)    
                    NMCVector = 0x00FF
                        
                elif len(Package) == 9:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses, Package[3].Synapses)
                    HYPERCOLUMN1_PreSynapticStream = Synapse2Stream(Package[4].Synapses , Package[5].Synapses, Package[6].Synapses, Package[7].Synapses)
                    HYPERCOLUMN2_PreSynapticStream = Synapse2Stream(Package[8].Synapses)                      
                    NMCVector = 0x01FF
                        
                elif len(Package) == 10:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses, Package[3].Synapses)
                    HYPERCOLUMN1_PreSynapticStream = Synapse2Stream(Package[4].Synapses , Package[5].Synapses, Package[6].Synapses, Package[7].Synapses)
                    HYPERCOLUMN2_PreSynapticStream = Synapse2Stream(Package[8].Synapses , Package[9].Synapses)  
                    NMCVector = 0x03FF
           
                elif len(Package) == 11:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses, Package[3].Synapses)
                    HYPERCOLUMN1_PreSynapticStream = Synapse2Stream(Package[4].Synapses , Package[5].Synapses, Package[6].Synapses, Package[7].Synapses)
                    HYPERCOLUMN2_PreSynapticStream = Synapse2Stream(Package[8].Synapses , Package[9].Synapses , Package[10].Synapses)
                    NMCVector = 0x07FF
                        
                elif len(Package) == 12:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses, Package[3].Synapses)
                    HYPERCOLUMN1_PreSynapticStream = Synapse2Stream(Package[4].Synapses , Package[5].Synapses, Package[6].Synapses, Package[7].Synapses)
                    HYPERCOLUMN2_PreSynapticStream = Synapse2Stream(Package[8].Synapses , Package[9].Synapses , Package[10].Synapses , Package[11].Synapses)
                    NMCVector = 0x0FFF
                        
                elif len(Package) == 13:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses, Package[3].Synapses)
                    HYPERCOLUMN1_PreSynapticStream = Synapse2Stream(Package[4].Synapses , Package[5].Synapses, Package[6].Synapses, Package[7].Synapses)
                    HYPERCOLUMN2_PreSynapticStream = Synapse2Stream(Package[8].Synapses , Package[9].Synapses , Package[10].Synapses , Package[11].Synapses)
                    HYPERCOLUMN3_PreSynapticStream = Synapse2Stream(Package[12].Synapses)                      
                    NMCVector = 0x1FFF
                        
                elif len(Package) == 14:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses, Package[3].Synapses)
                    HYPERCOLUMN1_PreSynapticStream = Synapse2Stream(Package[4].Synapses , Package[5].Synapses, Package[6].Synapses, Package[7].Synapses)
                    HYPERCOLUMN2_PreSynapticStream = Synapse2Stream(Package[8].Synapses , Package[9].Synapses , Package[10].Synapses , Package[11].Synapses)
                    HYPERCOLUMN3_PreSynapticStream = Synapse2Stream(Package[12].Synapses, Package[13].Synapses )    
                    NMCVector = 0x3FFF
                        
                elif len(Package) == 15:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses, Package[3].Synapses)
                    HYPERCOLUMN1_PreSynapticStream = Synapse2Stream(Package[4].Synapses , Package[5].Synapses, Package[6].Synapses, Package[7].Synapses)
                    HYPERCOLUMN2_PreSynapticStream = Synapse2Stream(Package[8].Synapses , Package[9].Synapses , Package[10].Synapses , Package[11].Synapses)
                    HYPERCOLUMN3_PreSynapticStream = Synapse2Stream(Package[12].Synapses, Package[13].Synapses , Package[14].Synapses)                     
                    NMCVector = 0x7FFF
                        
                elif len(Package) == 16:
                        
                    HYPERCOLUMN0_PreSynapticStream = Synapse2Stream(Package[0].Synapses , Package[1].Synapses, Package[2].Synapses, Package[3].Synapses)
                    HYPERCOLUMN1_PreSynapticStream = Synapse2Stream(Package[4].Synapses , Package[5].Synapses, Package[6].Synapses, Package[7].Synapses)
                    HYPERCOLUMN2_PreSynapticStream = Synapse2Stream(Package[8].Synapses , Package[9].Synapses , Package[10].Synapses , Package[11].Synapses)
                    HYPERCOLUMN3_PreSynapticStream = Synapse2Stream(Package[12].Synapses, Package[13].Synapses , Package[14].Synapses , Package[14].Synapses)   
                    NMCVector = 0xFFFF
                                                    
                if len(HYPERCOLUMN0_PreSynapticStream) != 0:
                    HP0_SynapticBuffer = allocate(shape=(len(HYPERCOLUMN0_PreSynapticStream),), dtype=np.uint64)
                    HP0_SynapticBufferPhyAddr = (HP0_SynapticBuffer.device_address) 
                    for i in range(len(HYPERCOLUMN0_PreSynapticStream)):
                        HP0_SynapticBuffer[i] = HYPERCOLUMN0_PreSynapticStream[i]
                    HP0_PostSynapticBuffer = allocate(shape=(len(HYPERCOLUMN0_PreSynapticStream)+5,), dtype=np.uint64)
                    HP0_PostSynapticBufferPhyAddr = (HP0_PostSynapticBuffer.device_address)         
       
                    mmio.write(PRE_SYNAPTIC_STREAM_ROUTER,0x1)
                    mmio.write(PRE_SYNAPTIC_STREAM_STROBE,0xF)
                    PushSynapses.transfer(HP0_SynapticBuffer)
                    PushSynapses.wait()   
                        
                if len(HYPERCOLUMN1_PreSynapticStream) != 0:                
                    HP1_SynapticBuffer = allocate(shape=(len(HYPERCOLUMN1_PreSynapticStream),), dtype=np.uint64)
                    HP1_SynapticBufferPhyAddr = (HP1_SynapticBuffer.device_address) 
                    for i in range(len(HYPERCOLUMN1_PreSynapticStream)):
                        HP1_SynapticBuffer[i] = HYPERCOLUMN1_PreSynapticStream[i]
                    HP1_PostSynapticBuffer = allocate(shape=(len(HYPERCOLUMN1_PreSynapticStream)+5,), dtype=np.uint64)
                    HP1_PostSynapticBufferPhyAddr = (HP1_PostSynapticBuffer.device_address) 

                    mmio.write(PRE_SYNAPTIC_STREAM_ROUTER,0x2)
                    mmio.write(PRE_SYNAPTIC_STREAM_STROBE,0xF)
                    PushSynapses.transfer(HP1_SynapticBuffer)
                    PushSynapses.wait()
                        
                if len(HYPERCOLUMN2_PreSynapticStream) != 0:                
                    HP2_SynapticBuffer = allocate(shape=(len(HYPERCOLUMN2_PreSynapticStream),), dtype=np.uint64)
                    HP2_SynapticBufferPhyAddr = (HP2_SynapticBuffer.device_address) 
                    for i in range(len(HYPERCOLUMN2_PreSynapticStream)):
                        HP2_SynapticBuffer[i] = HYPERCOLUMN2_PreSynapticStream[i]
                    HP2_PostSynapticBuffer = allocate(shape=(len(HYPERCOLUMN2_PreSynapticStream)+5,), dtype=np.uint64)
                    HP2_PostSynapticBufferPhyAddr = (HP2_PostSynapticBuffer.device_address) 

                    mmio.write(PRE_SYNAPTIC_STREAM_ROUTER,0x4)
                    mmio.write(PRE_SYNAPTIC_STREAM_STROBE,0xF)
                    PushSynapses.transfer(HP2_SynapticBuffer)
                    PushSynapses.wait()

                if len(HYPERCOLUMN3_PreSynapticStream) != 0:                                    
                    HP3_SynapticBuffer = allocate(shape=(len(HYPERCOLUMN3_PreSynapticStream),), dtype=np.uint64)
                    HP3_SynapticBufferPhyAddr = (HP3_SynapticBuffer.device_address)    
                    for i in range(len(HYPERCOLUMN3_PreSynapticStream)):
                        HP3_SynapticBuffer[i] = HYPERCOLUMN3_PreSynapticStream[i]                          
                    HP3_PostSynapticBuffer = allocate(shape=(len(HYPERCOLUMN3_PreSynapticStream)+5,), dtype=np.uint64)
                    HP3_PostSynapticBufferPhyAddr = (HP3_PostSynapticBuffer.device_address) 
                    
                    mmio.write(PRE_SYNAPTIC_STREAM_ROUTER,0x8)
                    mmio.write(PRE_SYNAPTIC_STREAM_STROBE,0xF)
                    PushSynapses.transfer(HP3_SynapticBuffer)
                    PushSynapses.wait()
                        
                mmio.write(PRE_SYNAPTIC_STREAM_ROUTER,0x0)
                mmio.write(PRE_SYNAPTIC_STREAM_STROBE,0xF)
                mmio.write(HYPERCOLUMN_INFERENCE_START,0xF)
                    
                InferenceStatus = mmio.read(HYPERCOLUMN_INFERENCE_DONE)
                while InferenceStatus == 0:
                    InferenceStatus = mmio.read(HYPERCOLUMN_INFERENCE_DONE)
                mmio.write(HYPERCOLUMN_INFERENCE_START,0x0)

                mmio.write(NMC_COLD_START,NMCVector) 

                NeuronStatus = mmio.read(NMC_FINISHED)
                while NeuronStatus == 0:
                    NeuronStatus = mmio.read(NMC_FINISHED)
  
                mmio.write(NMC_COLD_START,0x0)
                        
                for m in range(len(Package)):
                    mmio.write(NMC_POST_SYN_INTERFACE_CNTRL,(2**m))
                    mmio.write(POST_SYN_MUX,m)
                        
                    Package[m].NeuronID = mmio.read(NMC_XX_R_NMODEL_NEURON_ID)
                    Package[m].LastSpikeTime = mmio.read(NMC_XX_R_NNMODEL_NEW_SPIKE_TIME)
                    
                    Package[m].SynapticQFactor = mmio.read(NMC_XX_R_NMODEL_SYN_QFACTOR)
                    Package[m].NeuronRefractoryDuration = mmio.read(NMC_XX_R_NMODEL_REFRACTORY_DUR)
                        
                    if isinstance(Package[m].NeuronParamDataArray, list):
                        for i in range(len(Package[m].NeuronParamDataArray)):
                            mmio.write(NMC_XX_REDIST_NMODEL_DADDR,Package[m].NeuronParamAddrArray[i])
                            Package[m].NeuronParamDataArray[i] = mmio.read(NMC_XX_R_NMODEL_NPARAM_DATAOUT)
                            
                    else:
                        mmio.write(NMC_XX_REDIST_NMODEL_DADDR,Package[m].NeuronParamAddrArray)
                        Package[m].NeuronParamDataArray = mmio.read(NMC_XX_R_NMODEL_NPARAM_DATAOUT)
    
                mmio.write(NMC_POST_SYN_INTERFACE_CNTRL,0x0) 
        
                mmio.write(HYPERCOLUMN_LEARNING_START,0xF) 
                LearningStatus = mmio.read(HYPERCOLUMN_LEARNING_DONE)
                while LearningStatus == 0:
                    LearningStatus = mmio.read(HYPERCOLUMN_LEARNING_DONE)
                mmio.write(HYPERCOLUMN_LEARNING_START,0x0) 
                        
                tempspikes = mmio.read(NMC_SPIKE_FLAGS)
                PostSpikeStream.append(tempspikes)
                    
                mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x0)
                mmio.write(POST_SYNAPTIC_STREAM_STROBE,0xF)
                PostSynapticStream.register_map.S2MM_DMACR.RS = 0
                PostSynapticStream.register_map.S2MM_DMACR.Reset = 1
                PostSynapticStream.register_map.S2MM_DMACR.Reset = 0
                PostSynapticStream.register_map.S2MM_DMACR.RS = 1
                    
                if len(HYPERCOLUMN0_PreSynapticStream) != 0:
                        
                    mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x0)
                    mmio.write(POST_SYNAPTIC_STREAM_STROBE,0xF)
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 0
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 1
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 0
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 1
                
                    mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x1)
                    PostSynapticStream.register_map.S2MM_DA = HP0_PostSynapticBufferPhyAddr
                    PostSynapticStream.register_map.S2MM_LENGTH = (len(HYPERCOLUMN0_PreSynapticStream)+5)*8
    
                    mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x0)
                    mmio.write(POST_SYNAPTIC_STREAM_STROBE,0xF)
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 0
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 1
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 0
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 1
                        
                    HP0_SynapticBuffer = HP0_PostSynapticBuffer[5:(len(HYPERCOLUMN0_PreSynapticStream)+5)]              

                    for i in range(len(HP0_SynapticBuffer)):
                        HYPERCOLUMN0_PreSynapticStream[i] = HP0_SynapticBuffer[i]
                        
                    del HP0_SynapticBuffer
                        
                if len(HYPERCOLUMN1_PreSynapticStream) != 0:
                
                    mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x0)
                    mmio.write(POST_SYNAPTIC_STREAM_STROBE,0xF)
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 0
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 1
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 0
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 1
                
                    mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x2)
                    PostSynapticStream.register_map.S2MM_DA = HP1_PostSynapticBufferPhyAddr
                    PostSynapticStream.register_map.S2MM_LENGTH = (len(HYPERCOLUMN1_PreSynapticStream)+5)*8
                        
                    mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x0)
                    mmio.write(POST_SYNAPTIC_STREAM_STROBE,0xF)
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 0
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 1
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 0
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 1
                        
                    HP1_SynapticBuffer = HP1_PostSynapticBuffer[5:(len(HYPERCOLUMN1_PreSynapticStream)+5)]
                        
                    for i in range(len(HP1_SynapticBuffer)):
                        HYPERCOLUMN1_PreSynapticStream[i] = HP1_SynapticBuffer[i]
                        
                    del HP1_SynapticBuffer
                        
                if len(HYPERCOLUMN2_PreSynapticStream) != 0:
                
                    mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x0)
                    mmio.write(POST_SYNAPTIC_STREAM_STROBE,0xF)
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 0
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 1
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 0
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 1
                
                    mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x4)
                    PostSynapticStream.register_map.S2MM_DA = HP2_PostSynapticBufferPhyAddr
                    PostSynapticStream.register_map.S2MM_LENGTH = (len(HYPERCOLUMN2_PreSynapticStream)+5)*8
                        
                    mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x0)
                    mmio.write(POST_SYNAPTIC_STREAM_STROBE,0xF)
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 0
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 1
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 0
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 1
                        
                    HP2_SynapticBuffer = HP2_PostSynapticBuffer[5:(len(HYPERCOLUMN2_PreSynapticStream)+5)]
                        
                    for i in range(len(HP2_SynapticBuffer)):
                        HYPERCOLUMN2_PreSynapticStream[i] = HP2_SynapticBuffer[i]
                        
                    del HP2_SynapticBuffer
                        
                if len(HYPERCOLUMN3_PreSynapticStream) != 0:
                
                    mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x0)
                    mmio.write(POST_SYNAPTIC_STREAM_STROBE,0xF)
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 0
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 1
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 0
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 1
                
                    mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x8)
                    PostSynapticStream.register_map.S2MM_DA = HP3_PostSynapticBufferPhyAddr
                    PostSynapticStream.register_map.S2MM_LENGTH = (len(HYPERCOLUMN3_PreSynapticStream)+5)*8
                        
                    mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x0)
                    mmio.write(POST_SYNAPTIC_STREAM_STROBE,0xF)
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 0
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 1
                    PostSynapticStream.register_map.S2MM_DMACR.Reset = 0
                    PostSynapticStream.register_map.S2MM_DMACR.RS = 1
                        
                    HP3_SynapticBuffer = HP3_PostSynapticBuffer[5:(len(HYPERCOLUMN3_PreSynapticStream)+5)] 
                        
                    for i in range(len(HP3_SynapticBuffer)):
                        HYPERCOLUMN3_PreSynapticStream[i] = HP3_SynapticBuffer[i]
                        
                    del HP3_SynapticBuffer

                mmio.write(POST_SYNAPTIC_STREAM_ROUTER,0x0)
                mmio.write(POST_SYNAPTIC_STREAM_STROBE,0xF)
                PostSynapticStream.register_map.S2MM_DMACR.RS = 0
                PostSynapticStream.register_map.S2MM_DMACR.Reset = 1
                PostSynapticStream.register_map.S2MM_DMACR.Reset = 0
                PostSynapticStream.register_map.S2MM_DMACR.RS = 1   

                junk0=[]
                junk1=[]
                junk2=[]
                    
                if len(Package) == 1:
                        
                    Package[0].Synapses,junk0,junk1,junk2=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)                
                            
                elif len(Package) == 2:
                        
                    Package[0].Synapses,Package[1].Synapses,junk0,junk1=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)                
                        
                elif len(Package) == 3:
                        
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,junk0=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                        
                elif len(Package) == 4:
                        
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,Package[3].Synapses=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                        
                elif len(Package) == 5:
                        
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,Package[3].Synapses=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                    Package[4].Synapses,junk0,junk1,junk2=Stream2Synapse(HYPERCOLUMN1_PreSynapticStream)
                                                       
                elif len(Package) == 6:
                        
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,Package[3].Synapses=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                    Package[4].Synapses,Package[5].Synapses,junk0,junk1=Stream2Synapse(HYPERCOLUMN1_PreSynapticStream)
                    
                elif len(Package) == 7:
                        
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,Package[3].Synapses=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                    Package[4].Synapses,Package[5].Synapses,Package[6].Synapses,junk0=Stream2Synapse(HYPERCOLUMN1_PreSynapticStream)
    
                elif len(Package) == 8:
                        
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,Package[3].Synapses=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                    Package[4].Synapses,Package[5].Synapses,Package[6].Synapses,Package[7].Synapses=Stream2Synapse(HYPERCOLUMN1_PreSynapticStream)
                        
                elif len(Package) == 9:
                        
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,Package[3].Synapses=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                    Package[4].Synapses,Package[5].Synapses,Package[6].Synapses,Package[7].Synapses=Stream2Synapse(HYPERCOLUMN1_PreSynapticStream)
                    Package[8].Synapses,junk0,junk1,junk2=Stream2Synapse(HYPERCOLUMN2_PreSynapticStream)   
                        
                elif len(Package) == 10:
                        
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,Package[3].Synapses=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                    Package[4].Synapses,Package[5].Synapses,Package[6].Synapses,Package[7].Synapses=Stream2Synapse(HYPERCOLUMN1_PreSynapticStream)
                    Package[8].Synapses,Package[9].Synapses,junk0,junk1=Stream2Synapse(HYPERCOLUMN2_PreSynapticStream)
                        
                elif len(Package) == 11:
                      
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,Package[3].Synapses=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                    Package[4].Synapses,Package[5].Synapses,Package[6].Synapses,Package[7].Synapses=Stream2Synapse(HYPERCOLUMN1_PreSynapticStream)
                    Package[8].Synapses,Package[9].Synapses,Package[10].Synapses,junk0=Stream2Synapse(HYPERCOLUMN2_PreSynapticStream)
                        
                elif len(Package) == 12:
                        
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,Package[3].Synapses=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                    Package[4].Synapses,Package[5].Synapses,Package[6].Synapses,Package[7].Synapses=Stream2Synapse(HYPERCOLUMN1_PreSynapticStream)
                    Package[8].Synapses,Package[9].Synapses,Package[10].Synapses,Package[11].Synapses=Stream2Synapse(HYPERCOLUMN2_PreSynapticStream)
                        
                elif len(Package) == 13:
                        
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,Package[3].Synapses=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                    Package[4].Synapses,Package[5].Synapses,Package[6].Synapses,Package[7].Synapses=Stream2Synapse(HYPERCOLUMN1_PreSynapticStream)
                    Package[8].Synapses,Package[9].Synapses,Package[10].Synapses,Package[11].Synapses=Stream2Synapse(HYPERCOLUMN2_PreSynapticStream)
                    Package[12].Synapses,junk0,junk1,junk2=Stream2Synapse(HYPERCOLUMN3_PreSynapticStream)
                        
                elif len(Package) == 14:
                
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,Package[3].Synapses=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                    Package[4].Synapses,Package[5].Synapses,Package[6].Synapses,Package[7].Synapses=Stream2Synapse(HYPERCOLUMN1_PreSynapticStream)
                    Package[8].Synapses,Package[9].Synapses,Package[10].Synapses,Package[11].Synapses=Stream2Synapse(HYPERCOLUMN2_PreSynapticStream)
                    Package[12].Synapses,Package[13].Synapses,junk0,junk1=Stream2Synapse(HYPERCOLUMN3_PreSynapticStream) 
                        
                elif len(Package) == 15:
                                           
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,Package[3].Synapses=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                    Package[4].Synapses,Package[5].Synapses,Package[6].Synapses,Package[7].Synapses=Stream2Synapse(HYPERCOLUMN1_PreSynapticStream)
                    Package[8].Synapses,Package[9].Synapses,Package[10].Synapses,Package[11].Synapses=Stream2Synapse(HYPERCOLUMN2_PreSynapticStream)
                    Package[12].Synapses,Package[13].Synapses,Package[14].Synapses,junk0=Stream2Synapse(HYPERCOLUMN3_PreSynapticStream)
                        
                elif len(Package) == 16:
                        
                    Package[0].Synapses,Package[1].Synapses,Package[2].Synapses,Package[3].Synapses=Stream2Synapse(HYPERCOLUMN0_PreSynapticStream)
                    Package[4].Synapses,Package[5].Synapses,Package[6].Synapses,Package[7].Synapses=Stream2Synapse(HYPERCOLUMN1_PreSynapticStream)
                    Package[8].Synapses,Package[9].Synapses,Package[10].Synapses,Package[11].Synapses=Stream2Synapse(HYPERCOLUMN2_PreSynapticStream)
                    Package[12].Synapses,Package[13].Synapses,Package[14].Synapses,Package[15].Synapses=Stream2Synapse(HYPERCOLUMN3_PreSynapticStream)
                    
                mmio.write(PRE_SYN_FIFO_RST,0xFFFF)
                mmio.write(POST_SYN_FIFO_RST,0xFFFF)
                mmio.write(HYPERCOLUMN_RST,0xFFFF)
                mmio.write(NMC_STATE_RST,0xFFFF)
                mmio.write(DC_RST_VECTOR,0xFFFF)
                mmio.write(PRE_SYN_FIFO_RST,0x0)
                mmio.write(POST_SYN_FIFO_RST,0x0)
                mmio.write(HYPERCOLUMN_RST,0x0)
                mmio.write(NMC_STATE_RST,0x0)
                mmio.write(DC_RST_VECTOR,0x0)
                
            OutStream = PostSpikeStream

        return OutStream      

    def Train(self, epochs, dataset):
        DatasetSize = count_lines(dataset)
        #tqdm.write(f'Dataset contains {DatasetSize} lines.')
        # Getting Dataset Line Length
        with open(dataset, 'r') as file:
            line = file.readline()
        elements = line.strip().split(' ')
        num_elements = len(elements)
        #tqdm.write(f'1 Line = {num_elements} optimized and packed spikes.')
        VectorCount = int(num_elements / 16)
        #tqdm.write(f'1 Line = {VectorCount} spike vectors')
        #tqdm.write(f'Training process is starting.')
        for epochCount in range(epochs):

            with tqdm(total=DatasetSize, desc=f"Epoch {epochCount + 1}/{epochs}") as pbar:
                for k in range(DatasetSize):
                    DatasetLine = DatasetLine2SpikeArray(dataset, k)
                    InputSpikeCycle = int(self.input_size / 16)
                
                    for l in range(0, len(DatasetLine), InputSpikeCycle):
                        SpikeVector = DatasetLine[l:l + InputSpikeCycle]
                        NetworkOut = self.Evaluate(SpikeVector, EnableLearning=True)
                
                    pbar.update(1)
        #tqdm.write(f'Training process is completed.')

    def Run(self,timestep,Spikes,EnableLearning):
        
        for k in range (timestep):
            SpikeCycles = self.input_size/16
            SpikeCycles = round(SpikeCycles)
            SpikeCycles = int(SpikeCycles)
            
            if Spikes == 'Zeros':
                SpikeFeed = np.zeros(SpikeCycles, dtype=np.uint16)
            elif Spikes == 'Ones':
                SpikeFeed = np.full(SpikeCycles, 65535, dtype=np.uint16)
            elif Spikes == 'Random':
                SpikeFeed = np.random.randint(0, 2**16, size=SpikeCycles, dtype=np.uint16)
                
            NetworkOut = self.Evaluate(SpikeFeed,EnableLearning=EnableLearning)
            

In [ ]:
# Creating LearningWindow Input Arguments

TimewindowLeft = 50e-3
TimewindowRight = 50e-3
ZeroPoint= 0
Samples= 256
on_pre = '''0.6*exp(t/10e-3)'''
on_post = '''-0.6*exp(-t/10e-3)'''

BiphasicSTDP = Synapse(TimewindowLeft, TimewindowRight, ZeroPoint, Samples, on_pre, on_post)

In [ ]:
# Spike Response Model Assembly Code
# Address of Parameter 3C00 (FP16 1.0) = 59
# Address of Parameter Pmin  = 60
# Address of Parameter Pth   = 61
# Address of Parameter D     = 62
# Address of Parameter Pref  = 63
# Address of Parameter Rp    = 64
# Address of Parameter Pt-1 (Membran Voltage/State Variable) = 65
SRM_BASEADDR = int(256)
SRM = '''
lw,x0,59
lw,x2,60
lw,x3,61
lw,x4,62
lw,x5,63
lw,x6,64
lw,x7,65
comp,x7,x2
big,3
sw,x6,65
return
comp,x7,x3
bil,5
sw,x5,65
spk
strf,27
return
fmac,x7,x0
smac,x4,x0
getacc,x7
clracc
sw,x7,65
return
'''

In [ ]:
# NMC Neuron Parameter Pass
# NPARAM_DATA[31:16] & NPARAM_ADDR[15:0]

# Address of Parameter 3C00 (FP16 1.0) = 59
# Address of Parameter Pmin  = 60
# Address of Parameter Pth   = 61
# Address of Parameter D     = 62
# Address of Parameter Pref  = 63
# Address of Parameter Rp    = 64
# Address of Parameter Pt-1 (Membran Voltage/State Variable) = 65

# Value of Parameter Pmin      = -37 (D0A0)
# Value of Parameter Pth       = 77  (54D0)
# Value of Parameter D         = 3   (4200)
# Value of Parameter Prefract  = 2   (4000)
# Value of Parameter Rp        = 0   (0)
# Value of Parameter Pt-1 (Membran Voltage/State Variable) = 65

SRMProgFlow = NModelAssembler(SRM,SRM_BASEADDR)
LoadNeuronModel(SRMProgFlow)
SRMProgFlowLow = SRM_BASEADDR
SRMParamData = int(0)
SRMParamAddr = int(65)

NMCParamPass(0x3C00,0x003B,0xFFFF)
NMCParamPass(0xD0A0,0x003C,0xFFFF)
NMCParamPass(0x54D0,0x003D,0xFFFF)
NMCParamPass(0x4200,0x003E,0xFFFF)
NMCParamPass(0x4000,0x003F,0xFFFF)
NMCParamPass(0x0000,0x0040,0xFFFF)
NMCParamPass(0x0000,0x0041,0xFFFF)

In [ ]:
# Network Run

# Initializing Converted and Packed Dataset
#file_name = './semeion/semeion_streamdata.dat'
#file_name = './semeion/semeion_streamdata10.dat'
file_name = './semeion/semeion_streamdata_full_10.dat'


In [ ]:
SNN = Network(input_size=256)
SNN.Append(NMODELLowAddr=SRMProgFlowLow, ParamData=SRMParamData, ParamAddr=SRMParamAddr, SynapseType=BiphasicSTDP, LearningRate=0.98, LayerSize = 16, Scheme='fc', Probability=None)


In [ ]:
SNN.Train(epochs=2,dataset=file_name)

In [ ]:
# TEST FONKSIYONU GELSIN
# DATASET COMPILE ISINI CIDDIYE ALMAK LAZIM